In [1]:
# _files = ["ae.m3u", "sy.m3u", "sa.m3u"]
import requests

url = "https://api.github.com/repos/iptv-org/iptv/contents/streams"

response = requests.get(url)
if response.status_code == 200:
    files = response.json()
    for file_info in files:
        # if file_info["name"] in _files:
        file_url = file_info["download_url"]
        file_response = requests.get(file_url)
        if file_response.status_code == 200:
            # print(f"Contents of {file_info['name']}:")
            for line in file_response.text.splitlines():
                if line.find('#EXTM3U') == -1 and line.find('EXTVLCOPT') == -1:
                    line = line.strip()
                    # print(line)
                    with open('t.txt', 'a', encoding='utf-8') as txtFile:
                        txtFile.write(line + '\n')
                        txtFile.close()
            # print('+++++++++', "\n" + "=" * 40 + "\n")
        # else:
        #     print(f"Failed to fetch {file_info['name']}")
else:
    print("Failed to retrieve file list:", response.status_code)

In [2]:
import pandas as pd
line_open = False
channels_dict = dict()
channel_url, channel_name = "", ""
with open("t.txt", "r", encoding='utf-8') as tFile:
    for i, line in enumerate(tFile.readlines()):
        line = line.replace("\n", "")
        # print(line)
        if line.find('#EXTINF:-1 tvg-id="') != -1:
            line_open = True
            channel_name = line[line.find(",") + 1 :]
            # print(channel_name)
        if line_open and line.find("http") != -1:
            channel_url = line
            line_open = False
            # print(f"{channel_name},{channel_url}")
            channels_dict[channel_name] = channel_url
df = pd.DataFrame({
    'channel_name':channels_dict.keys(),
    'channel_url':channels_dict.values()
})

In [3]:

channels = set()
with open('channels.m3u','r') as live_channels:
    for i, channel in enumerate(live_channels.readlines()):
        if i % 2 != 0:
            channel = channel.replace('\n','')
            channel = channel[channel.find(',') + 1:]
            channels.add(channel)
live_channels.close()

new_channels_list = df[df['channel_name'].isin(channels)].sort_values(by='channel_name', ascending=True)

import os
os.remove('channels_old.m3u')
os.rename('channels.m3u', 'channels_old.m3u')


def log_that_line(logWhat):
    try:
        file = open('channels.m3u', "a")
        file.write(logWhat)             
        file.close()
    except:
        log_that_line(logWhat)

log_that_line('#EXTM3U\n')
for i, row in new_channels_list.iterrows():
    name = row.iloc[0]
    url = row.iloc[1]
    line1 = '#EXTINF:-1 tvg-id="",' + name
    line2 = url
    log_that_line(line1 + '\n' + line2 + '\n')

os.remove('t.txt')
print('Channels urls updated!')

missing_channels = channels.difference(df['channel_name'].unique())
print('Missing Channels: \n', missing_channels)

Channels urls updated!
Missing Channels: 
 {'PowerTÃ¼rk TV (720p)'}
